### Importing all the libraries

In [49]:
from flask import Flask,request,render_template,jsonify
import os
import cv2
import random
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import librosa
import joblib
from sklearn.preprocessing import PowerTransformer
from moviepy import AudioFileClip
from moviepy import VideoFileClip
from sklearn.preprocessing import StandardScaler,MinMaxScaler

### Configuring flask app

In [50]:
app=Flask(__name__)
app.config['UPLOAD_FOLDER']='uploads'

In [51]:
os.makedirs(app.config['UPLOAD_FOLDER'],exist_ok=True)

In [52]:
@app.route('/')
def index():
    return render_template('upload.html')

### Functions and preprocessing for input of image model

In [53]:
def extract_frames_and_labels(video_path, frame_count, img_size=(224, 224)):
    X_data = []
    

    video = cv2.VideoCapture(video_path)

    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < frame_count:
        print(f"Video too short to extract {frame_count} frames.")
        video.release()
        return np.array([]), np.array([])
    step_size = total_frames // frame_count
    for i in range(frame_count):
        frame_index = i * step_size
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = video.read()
        if ret:
            frame = cv2.resize(frame, img_size)
            X_data.append(frame)

    
    frames = []
    for frame in X_data:
        frame = frame / 255.0
        frames.append(frame)
    video.release()

    return np.array([frames])

### Functions and preprocessing of input for audio model

In [54]:
def extract_audio(video_path, output_audio_path):
    video_clip = VideoFileClip(video_path)
    flag = True
    if video_clip.audio is not None:
        audio_clip = video_clip.audio
        audio_clip.write_audiofile(output_audio_path)
        audio_clip.close()
        print("Audio extracted and saved.")
    else:
        flag = False
        print("No audio track found in the video.")

    video_clip.close()
    return flag
    
def extract_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)
    os.remove(audio_path)
    features = {    }
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    
    selected_mfcc_indices = [3] + list(range(5, 21))
    for i in selected_mfcc_indices:
        features[f'mfcc{i}'] = np.mean(mfccs[i - 1])
    
    return features

In [55]:
from flask import render_template

@app.route('/upload', methods=['POST'])
def upload_video():
    if 'video' not in request.files:
        return render_template('upload.html', error="No video is found")
    
    video = request.files['video']
    if video.filename == '':
        return render_template('upload.html', error="No selected video")
    
    if video:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], video.filename)
        video.save(filepath)
        data = extract_frames_and_labels(filepath, 32)
        model_image = load_model("FramesModel.h5")
        prediction_video = model_image.predict(data)
        audio_path = "extracted_audio.wav"
        flag = extract_audio(filepath, audio_path)

        if flag:
            features = extract_features(audio_path)
            columns = list(features.keys())
            values = list(features.values())
            audio_data = pd.DataFrame([values], columns=columns)
            pt=PowerTransformer()
            audio_data['mfcc3']=pt.fit_transform(audio_data[['mfcc3']])
            scaler=joblib.load('Scaler.pkl')
            audio_data.iloc[:, :] = scaler.transform(audio_data)
            audio_model = load_model("new_sigmoid.h5")
            prediction_audio = audio_model.predict(audio_data)

       
        response = {
            "combined_result": None,
            "final_label": None
        }
        if prediction_audio is not None:
            result = prediction_video * 0.5 + prediction_audio*0.5
            response["combined_result"] = float(result) 
            response["final_label"] = "1 (Deepfake)" if result >0.5 else "0 (Not Deepfake)"
        else:
            result = prediction_video
            response["combined_result"] = float(result) 
            response["final_label"] = "1 (Deepfake)" if result >0.5 else "0 (Not Deepfake)"
            
        print(prediction_video)
        print(prediction_audio)
        return render_template('upload.html', response=response)

    return render_template('upload.html', error="Unexpected error occurred")


In [56]:
if __name__ =='__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2025 22:56:06] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 3s 3s/step
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Audio extracted and saved.
1/1 [==============================] - 0s 114ms/step


C:\Users\ishaa\AppData\Local\Temp\ipykernel_8620\80476574.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  response["combined_result"] = float(result)  # Cast to native float
127.0.0.1 - - [17/Jan/2025 22:56:32] "POST /upload HTTP/1.1" 200 -


[[0.2734]]
[[0.20858487]]
1/1 [==============================] - 4s 4s/step
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Audio extracted and saved.
1/1 [==============================] - 0s 144ms/step


C:\Users\ishaa\AppData\Local\Temp\ipykernel_8620\80476574.py:46: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  response["combined_result"] = float(result)  # Cast to native float
127.0.0.1 - - [17/Jan/2025 22:57:52] "POST /upload HTTP/1.1" 200 -


[[0.1632]]
[[0.34082747]]
